In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [13]:
#!pip install statsmodels

In [14]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# import lazypredict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

import statsmodels.tsa.stattools as st
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

In [15]:
df = pd.read_csv('GCB2022v27_MtCO2_flat.csv')
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [17]:
#dropping years less than 1950
df.drop(df[df['Year'] < 1950].index, inplace = True)

In [18]:
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
200,Afghanistan,AFG,1950,0.084272,0.021068,0.063204,0.000000,0.000000,0.000000,NaN,0.011266
201,Afghanistan,AFG,1951,0.091600,0.025648,0.065952,0.000000,0.000000,0.000000,NaN,0.012098
202,Afghanistan,AFG,1952,0.091600,0.031708,0.059892,0.000000,0.000000,0.000000,NaN,0.011946
203,Afghanistan,AFG,1953,0.106256,0.037949,0.068307,0.000000,0.000000,0.000000,NaN,0.013685
204,Afghanistan,AFG,1954,0.106256,0.042502,0.063754,0.000000,0.000000,0.000000,NaN,0.013511
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [21]:
#converting countries to list
countries=list(df["Country"].unique())

Feature Engineering

We would be filling missing data with the mean . The problem with using pandas to fill mean would be that it would only fill the mean of the entire column. For example, for the missing data in afganistan, it would fill from the mean of the countries which includes developed countries such as USA, China etc. To work Around this, we would be grouping the data by the mean for each country and then fill the missing data.

In [22]:
#Getting the mean of Each country
coal_mean=df.groupby('Country')['Coal'].mean()
oil_mean=df.groupby('Country')['Oil'].mean()
gas_mean=df.groupby('Country')['Gas'].mean()
cement_mean=df.groupby('Country')['Cement'].mean()
flaring_mean=df.groupby('Country')['Flaring'].mean()
other_mean=df.groupby('Country')['Other'].mean()
per_capita_mean= df.groupby('Country')['Per Capita'].mean()
#Setting the Index
coal_mean.index = countries
oil_mean.index = countries
gas_mean.index = countries
cement_mean.index = countries
flaring_mean.index = countries
other_mean.index = countries
per_capita_mean.index = countries
#Converting to Dictionary
coal_mean=coal_mean.to_dict()
oil_mean=oil_mean.to_dict()
gas_mean=gas_mean.to_dict()
cement_mean= cement_mean.to_dict()
flaring_mean=flaring_mean.to_dict()
other_mean=other_mean.to_dict()
per_capita_mean= per_capita_mean.to_dict()

In [23]:
#fillin missing values with the mean of that country and the type of fuel
df.Coal = df.Coal.fillna(df.Country.map(coal_mean))
df.Oil = df.Oil.fillna(df.Country.map(oil_mean))
df.Gas = df.Gas.fillna(df.Country.map(gas_mean))
df.Cement = df.Cement.fillna(df.Country.map(cement_mean))
df.Flaring = df.Flaring.fillna(df.Country.map(flaring_mean))
df.Other = df.Other.fillna(df.Country.map(other_mean))
df['Per Capita'].fillna(df.Country.map(per_capita_mean), inplace=True)

In [30]:
df.reset_index()
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
200,Afghanistan,AFG,1950,0.084272,0.021068,0.063204,0.000000,0.000000,0.000000,NaN,0.011266
201,Afghanistan,AFG,1951,0.091600,0.025648,0.065952,0.000000,0.000000,0.000000,NaN,0.012098
202,Afghanistan,AFG,1952,0.091600,0.031708,0.059892,0.000000,0.000000,0.000000,NaN,0.011946
203,Afghanistan,AFG,1953,0.106256,0.037949,0.068307,0.000000,0.000000,0.000000,NaN,0.013685
204,Afghanistan,AFG,1954,0.106256,0.042502,0.063754,0.000000,0.000000,0.000000,NaN,0.013511
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [31]:
mean_value=df['Other'].mean()
  
# Replace NaNs in column S2 with the
# mean of values in the same column
df['Other'].fillna(value=mean_value, inplace=True)
df.isna().sum()

Country                 0
ISO 3166-1 alpha-3    432
Year                    0
Total                   0
Coal                    0
Oil                     0
Gas                     0
Cement                  0
Flaring                 0
Other                   0
Per Capita              0
dtype: int64

In [32]:
#removing the values of the world
glob= df[df['ISO 3166-1 alpha-3'] == 'WLD']

In [33]:
#droppin missing iso values columns
df.drop(df[df['ISO 3166-1 alpha-3'] == 'WLD'].index, inplace = True)

In [34]:
#top 100 countries in each fuel category
Coal_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Coal'].mean().sort_values(ascending=False)[0:100].reset_index()
Oil_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Oil'].mean().sort_values(ascending=False)[0:100].reset_index()
Gas_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Gas'].mean().sort_values(ascending=False)[0:100].reset_index()
Cement_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Cement'].mean().sort_values(ascending=False)[0:100].reset_index()
Flaring_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Flaring'].mean().sort_values(ascending=False)[0:100].reset_index()
Other_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Other'].mean().sort_values(ascending=False)[0:100].reset_index()
Total_mean=df.groupby(['ISO 3166-1 alpha-3','Country'])['Total'].mean().sort_values(ascending=False)[0:100].reset_index()

In [35]:
categories=['Coal', 'Oil', 'Gas',
       'Cement', 'Flaring', 'Other','Total']
len(categories)

7

In [36]:
df_cat= ['Coal_mean','Oil_mean','Gas_mean','Cement_mean','Flaring_mean','Other_mean','Total_mean']

In [37]:
def chlorepath(data,color):
    fig=px.choropleth(data, locations="ISO 3166-1 alpha-3",
                    color= color, 
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Plasma)
    fig.show()

In [38]:
#getting the top 10 countries in each category
Coal_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Coal'].mean().sort_values(ascending=False)[0:10].reset_index()
Oil_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Oil'].mean().sort_values(ascending=False)[0:10].reset_index()
Gas_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Gas'].mean().sort_values(ascending=False)[0:10].reset_index()
Cement_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Cement'].mean().sort_values(ascending=False)[0:10].reset_index()
Flaring_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Flaring'].mean().sort_values(ascending=False)[0:10].reset_index()
Other_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Other'].mean().sort_values(ascending=False)[0:10].reset_index()
Total_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Total'].mean().sort_values(ascending=False)[0:10].reset_index()
Per_Capita_mean_10=df.groupby(['ISO 3166-1 alpha-3','Country'])['Per Capita'].mean().sort_values(ascending=False)[0:10].reset_index()
Year_mean_10_pc=df.groupby(['Year'])['Per Capita'].mean().sort_values(ascending=False)[0:10].reset_index()
Year_mean_10_total=df.groupby(['Year'])['Total'].mean().sort_values(ascending=False)[0:10].reset_index()

In [39]:
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
200,Afghanistan,AFG,1950,0.084272,0.021068,0.063204,0.0,0.000000,0.0,11.207946,0.011266
201,Afghanistan,AFG,1951,0.091600,0.025648,0.065952,0.0,0.000000,0.0,11.207946,0.012098
202,Afghanistan,AFG,1952,0.091600,0.031708,0.059892,0.0,0.000000,0.0,11.207946,0.011946
203,Afghanistan,AFG,1953,0.106256,0.037949,0.068307,0.0,0.000000,0.0,11.207946,0.013685
204,Afghanistan,AFG,1954,0.106256,0.042502,0.063754,0.0,0.000000,0.0,11.207946,0.013511
...,...,...,...,...,...,...,...,...,...,...,...
62827,International Transport,XIT,2017,1230.506945,1.665252,1230.506945,0.0,0.238179,0.0,11.207946,0.636299
62828,International Transport,XIT,2018,1270.690768,1.665252,1270.690768,0.0,0.238179,0.0,11.207946,0.636299
62829,International Transport,XIT,2019,1249.558280,1.665252,1249.558280,0.0,0.238179,0.0,11.207946,0.636299
62830,International Transport,XIT,2020,938.508679,1.665252,938.508679,0.0,0.238179,0.0,11.207946,0.636299
